<a href="https://colab.research.google.com/github/IlyasNasirov/notebook/blob/main/11_MA%CA%BCLUMOTLARNI_TAYYORLASH_11_28_YAKUNIY_AMALIY_VAZIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/h4PPC71.png)
#11-modul. Maʼlumotlarga ishlov berish. Maʼlumotlarni tayyorlash
# 11.28. Modul boʻyicha yakuniy amaliy vazifa

**Bu vazifalarni bajargach, maʼlumotlarni yuklash, tozalash, oʻzgartirish, matnlarni normalizatsiya qilish, qiymatlarni tekshirish, guruhlash va tahlil qilish kabi maʼlumotlarga ishlov berish (data processing) koʻnikmalarini egallaysiz.**

#**Umumiy maʼlumotlar bilan ishlash**

#Dataset haqida qisqacha maʼlumot
Restoran savdosi boʻyicha maʼlumotlar toʻplami 17 534 tranzaksiya uchun maʼlumotlarni oʻz ichiga oladi. Maʼlumotlar yetishmayotgan yoki baʼzilari toʻliq kiritilmagan boʻlishi mumkin. Maʼlumotlar toʻplamida asosiy taomlar, shirinliklar, ichimliklar kabi bir nechta toifadagi savdo tafsilotlari mavjud.


 0.   **Unnamed 0** – indekslarni;
 1.   **Order ID** – buyurtma IDʼsini;
 2.   **Customer ID** – mijoz IDʼsini;
 3.   **Category** – mahsulotlar kategoriyasini;
 4.   **Item** – mahsulot nomini;
 5.   **Price** – mahsulotning narxini;
 6.   **Quantity** – qancha mahsulot sotilgani miqdorini;
 7.   **Order Total** – umumiy narx qancha ekanini;
 8.   **Order Date** – buyurtma berilgan sanani (vaqt);
 9.   **Payment Method** – toʻlov usuli turini saqlaydi.

#1) Quyida berilgan datasetni yuklab olib, ustun nomlarini koʻring va ularni maʼno jihatdan tushunarli shaklga keltiring. `Unnamed: 0` ustunini tashlab yuboring!

In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/alishermutalov/praktikum-datasets/refs/heads/praktikum/restaurant_sales.csv"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,0,ORD_705844,CUST_092,NaN,side salad#?,3.0,1.0,3.0,2023-12-21,Credit Card
1,1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,3,ORD_630508,CUST_075,Drinks,NaN,-1.0,2.0,5.0,2022-08-09,Credit Card
4,4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,NaN,side salad#?,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,-1.0,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


#2) Maʼlumotlarning umumiy holatini (info(), describe()) tekshiring.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        17534 non-null  object 
 1   Customer ID     17534 non-null  object 
 2   Category        17498 non-null  object 
 3   Item            15776 non-null  object 
 4   Price           17534 non-null  float64
 5   Quantity        17104 non-null  float64
 6   Order Total     17104 non-null  float64
 7   Order Date      17534 non-null  object 
 8   Payment Method  16452 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.2+ MB


In [6]:
df.describe()

,Price,Quantity,Order Total
count,17534.000000,17104.000000,17104.000000
mean,6.207312,3.014149,19.914494
std,4.993782,1.414598,18.732549
min,-1.000000,1.000000,1.000000
25%,3.000000,2.000000,7.500000
50%,5.000000,3.000000,15.000000
75%,7.000000,4.000000,25.000000
max,20.000000,5.000000,100.000000


#3) Har bir ustundagi boʻsh qiymatlarni aniqlang va ularni toʻldirish strategiyasini ishlab chiqing.

In [52]:
df.isna().sum()
# "Item" ustunni bosh bolgan qiymatlrni unga togri keladigan "Category" boyicha toldiramiz.
# "Category" ustuni teskarisi "Item" ga qarab tuldirishimiz mumkin
# "Quantity" ustunini "Price" va 'Order Total' ga qarab topishimiz mumkin
# "Order Total" ham shunga ohshash, yaniyki  "Price" va "Quatitiy"ni kopaytiramiz
# "Payment Method" eng kop uchraydigan bilan toldirsak boladi

,0
Order ID,0
Customer ID,0
Category,36
Item,1758
Price,0
Quantity,430
Order Total,430
Order Date,0
Payment Method,1082



#4) Har bir ustundagi takroriy qiymatlarni toping va ularga qanday oʻzgartirish kiritish kerakligini hal qiling.

In [62]:
df.duplicated().sum()

np.int64(0)

#5) Maʼlumotlar toʻplamidagi noto‘g‘ri maʼlumotlar (masalan, son turishi kerak bo‘lgan ustunda matn bo‘lsa) mavjudligini tekshiring va to‘g‘rilang.

In [63]:
df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,NaN,side salad#?,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,-1.0,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        17534 non-null  object        
 1   Customer ID     17534 non-null  object        
 2   Category        17498 non-null  object        
 3   Item            15776 non-null  object        
 4   Price           17534 non-null  int64         
 5   Quantity        17534 non-null  int64         
 6   Order Total     17104 non-null  float64       
 7   Order Date      17534 non-null  datetime64[ns]
 8   Payment Method  16452 non-null  object        
 9   item            15776 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 1.3+ MB


In [67]:
df['Price'] = df['Price'].astype('int64')

In [73]:
df['Quantity'] = df['Quantity'].fillna(0).astype('int64')

In [82]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

#6) Maʼlumotlar toʻplamidagi **katta/kichik harf** muammolari boʻlgan joylarni toping va bir xil formatga keltiring.

In [99]:
df['Item'].unique()

array(['Side salad#?', 'Mashed potatoes', 'Grilled chicken', nan,
       'Pasta alfredo', 'Salmon', 'Garlic bread', 'Vegetarian platter',
       'Brownie', 'Sweet potato fries', 'Lemonade', 'Side salad',
       'Orange juice', 'Chocolate cake', 'Coca cola',
       'Grilled vegetables', 'Cheese fries', 'Beef chili', 'Chicken melt',
       'Fruit salad', 'French fries', 'Ice cream', 'Cheesecake',
       'Iced tea', 'Water (0.5)', 'Grilled vegetables?', 'Nachos grande#',
       'Water', 'Onion rings', 'Steak', 'Nachos grande',
       'Grilled chicken?', 'Chicken melt#', 'Lemonade (1.0)',
       'Grilled chicken#', 'Coca cola?', 'Steak#', 'Orange juice#',
       'French fries?', 'Side salad?', 'French fries#', 'Nachos grande?',
       'Brownie#', 'Chocolate cake#', 'Pasta alfredo#', 'Chicken melt?',
       'Ice cream#', 'Sweet potato fries?', 'Iced tea?',
       'Vegetarian platter?', 'Water#', 'Cheese fries?',
       'Grilled vegetables#', 'Beef chili#', 'Side salad#',
       'Mashed pota

In [98]:
df['Item'] = df['Item'].str.lower()
df['Item'] = df['Item'].str.capitalize()

#7) Maʼlumotlar turini oʻzgartirish.
 * Ustunlarning maʼlumot turlarini tekshiring. Notoʻgʻri turlarga ega boʻlgan ustunlarni toʻgʻri formatga oʻtkazing.
 * `order_date` yoki unga oʻxshash ustunni datetime formatiga oʻtkazing.

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        17534 non-null  object        
 1   Customer ID     17534 non-null  object        
 2   Category        17498 non-null  object        
 3   Item            15776 non-null  object        
 4   Price           17534 non-null  int64         
 5   Quantity        17534 non-null  int64         
 6   Order Total     17104 non-null  float64       
 7   Order Date      17534 non-null  datetime64[ns]
 8   Payment Method  16452 non-null  object        
 9   item            15776 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 1.3+ MB


#8) Mahsulot nomlarida noto‘g‘ri yoki nomaʼlum belgilar (?, NaN, #) mavjudligini tekshiring va ularni tuzating.

In [109]:
df.Item.unique()

array(['Side salad', 'Mashed potatoes', 'Grilled chicken', nan,
       'Pasta alfredo', 'Salmon', 'Garlic bread', 'Vegetarian platter',
       'Brownie', 'Sweet potato fries', 'Lemonade', 'Orange juice',
       'Chocolate cake', 'Coca cola', 'Grilled vegetables',
       'Cheese fries', 'Beef chili', 'Chicken melt', 'Fruit salad',
       'French fries', 'Ice cream', 'Cheesecake', 'Iced tea',
       'Water (0.5)', 'Nachos grande', 'Water', 'Onion rings', 'Steak',
       'Lemonade (1.0)'], dtype=object)

In [106]:
df.Item.isna().value_counts()

,count
Item,
False,15776
True,1758


In [108]:
df['Item'] = df['Item'].str.replace(r'\?|\#','',regex=True).str.strip()

In [128]:
df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method,item
0,ORD_705844,CUST_092,NaN,Side salad,3,1,3.0,2023-12-21,Credit Card,side salad#?
1,ORD_338528,CUST_021,Side Dishes,Mashed potatoes,4,3,12.0,2023-05-19,Digital Wallet,mashed potatoes
2,ORD_443849,CUST_029,Main Dishes,Grilled chicken,15,4,60.0,2023-09-27,Credit Card,grilled chicken
3,ORD_630508,CUST_075,Drinks,NaN,-1,2,5.0,2022-08-09,Credit Card,NaN
4,ORD_648269,CUST_031,Main Dishes,Pasta alfredo,12,4,48.0,2022-05-15,Cash,pasta alfredo


In [134]:
df['Item'].isna().value_counts()

,count
Item,
False,16658
True,876


In [133]:
fill_values = (
    df.groupby(['Category', 'Price'])['Item']
    .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)

df['Item'] = df['Item'].fillna(fill_values)

#**Maʼlumotlarni filtrlash va qayta ishlash**

#9) Manfiy narx yoki 0 ga teng narx qiymatlarini toping va ularni qanday oʻzgartirish kerakligini hal qiling.

In [140]:
df['Price'] = df['Price'].abs()

#10) Boʻsh toifalar (kategoriya) mavjudligini tekshiring va toʻgʻri kategoriya berish strategiyasini ishlab chiqing.

In [153]:
df['Item'] = df.groupby('Category')['Item'].transform(lambda x: x.ffill().bfill())

#11) Bir xil, lekin turlicha yozilgan qiymatlar (Coffee, cofee, COFEE) mavjudligini tekshiring va tuzating.

In [164]:
df['Item'].unique()

array([nan, 'Mashed potatoes', 'Grilled chicken', 'Lemonade',
       'Pasta alfredo', 'Salmon', 'Garlic bread', 'Chocolate cake',
       'Vegetarian platter', 'Brownie', 'Sweet potato fries',
       'Side salad', 'Fruit salad', 'Orange juice', 'Coca cola',
       'Grilled vegetables', 'Cheese fries', 'Beef chili', 'Chicken melt',
       'French fries', 'Ice cream', 'Cheesecake', 'Iced tea', 'Water',
       'Nachos grande', 'Onion rings', 'Steak'], dtype=object)

#12) `Customer ID` yoki boshqa identifikatorlarning takrorlanishini tekshiring.

In [162]:
df['Customer ID'].duplicated().value_counts()

,count
Customer ID,
True,17434
False,100


(#13) Mahsulotning ortiqcha taʼriflarini olib tashlang (Water (0.5), Water (1.0), Water → faqat Water qoldirish).

In [163]:
df['Item'] = df['Item'].str.replace(r"\s*\([^)]*\)", "", regex=True)

#**Qayta ishlangan maʼlumotlar bilan tahlil**

#14) Eng koʻp sotilgan 5 ta mahsulotni aniqlang.

In [166]:
df.groupby('Item')['Quantity'].sum().sort_values(ascending=False).head(5)

,Quantity
Item,
Pasta alfredo,3371
Side salad,3198
Ice cream,3169
Water,3160
French fries,2901


#15) Har bir mijoz oʻrtacha qancha pul sarflaganini hisoblang.

In [173]:
df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method,item
0,ORD_705844,CUST_092,NaN,NaN,3,1,3.0,2023-12-21,Credit Card,side salad#?
1,ORD_338528,CUST_021,Side Dishes,Mashed potatoes,4,3,12.0,2023-05-19,Digital Wallet,mashed potatoes
2,ORD_443849,CUST_029,Main Dishes,Grilled chicken,15,4,60.0,2023-09-27,Credit Card,grilled chicken
3,ORD_630508,CUST_075,Drinks,Lemonade,1,2,5.0,2022-08-09,Credit Card,NaN
4,ORD_648269,CUST_031,Main Dishes,Pasta alfredo,12,4,48.0,2022-05-15,Cash,pasta alfredo


In [183]:
df.groupby("Customer ID")['Order Total'].sum().mean()

np.float64(3406.175)

#16) Mijozlarning qaysi kunlarda koʻproq xarid qilgani bo‘yicha tahlil qiling.

In [187]:
df.groupby('Order Date')['Quantity'].sum()

,Quantity
Order Date,
2022-01-01,90
2022-01-02,76
2022-01-03,67
2022-01-04,94
2022-01-05,62
...,...
2023-12-27,90
2023-12-28,69
2023-12-29,77


#17) Narxi oʻrtacha narxdan ikki barobar yuqori boʻlgan mahsulotlarni toping.

In [193]:
average_price = df['Price'].mean()
print(average_price)
df[df['Price'] > 2 * average_price]['Item']

6.27460933044371


,Item
2,Grilled chicken
5,Salmon
7,Grilled chicken
9,Vegetarian platter
11,Salmon
...,...
17453,Salmon
17480,Salmon
17490,Salmon
17495,Steak


#18) Mahsulotlar kategoriyalari bo‘yicha tahlil qiling va eng ko‘p sotiladigan kategoriyalarni aniqlang.

In [195]:
df.groupby('Category')['Quantity'].sum().sort_values(ascending=False)

,Quantity
Category,
Main Dishes,10533
Starters,10317
Desserts,10234
Drinks,10226
Side Dishes,10147


#19) Har bir mijozning oʻrtacha xarid miqdorini (`groupby("customer_id")`) hisoblang.

In [199]:
df.groupby('Customer ID')['Order Total'].mean()

,Order Total
Customer ID,
CUST_001,17.784483
CUST_002,19.904070
CUST_003,17.200637
CUST_004,19.503012
CUST_005,17.918301
...,...
CUST_096,18.752604
CUST_097,19.662983
CUST_098,20.093220


#20) Qanday omillar sotuv hajmiga ta’sir qiladi? Bu bo‘yicha umumiy xulosa chiqaring. Javobni quyida kommentariya sifatida yozing!

In [ ]:
# Eng ko'p sotilgan mahsulotlar: Pasta alfredo, Side salad, Ice cream, Water, French fries kabi mahsulotlar yuqori talabga ega va sotuv hajmini oshiradi.
# Mijozlarning o'rtacha xarid miqdori.Har bir mijozning o'rtacha sarflagan puli umumiy sotuvga ta'sir qiladi.
# Kunlik sotuvlar. Malum kunlarda (dam olish va bayramlarda) sotuvlar hajmi yuqori bolishi mumkin.
# Mahsulot kategoriyalari. Bazi kategoriyalar (Main Dishes, Starters) boshqalarga qaraganda ko'proq sotiladi. Eng mashhur kategoriyalarga e'tibor qaratilsa sotuvlarni oshiradi.
# O'rtacha narxdan yuqori bo'lgan mahsulotlar ham o'z xaridorlariga ega, bu ham umumiy sotuv hajmiga o'z ta'sirini ko'rsatadi.